In [1]:
%matplotlib inline
!pip install gurobipy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import gurobipy as gp
from gurobipy import GRB
from time import time


def read_data():
    data = pd.read_csv('constants.csv')
    P = data['value'][0]
    R = data['value'][1]
    n = data['value'][2]
    return (P,R,n)

def instance_data(number_of_instance):
    instance = pd.read_csv('instance_'+str(number_of_instance)+'.csv')
    m = instance.shape[0]
    p = []
    r = []
    for p_i in instance['p_i']:
        p.append(p_i)
    for r_i in instance['r_i']:
        r.append(r_i)
    return (m,p,r)

def Model( number_of_instance , m , p , r , P , R , n ):
    global model, x_vars , y_vars , run_time
    model = gp.Model("q1_instance_"+str(number_of_instance))
    x_vars = {}
    y_vars = {}
    start = time()
    for i in range(n):
        for j in range(m):
            x_vars[i,j] = 0 
    for i in range(m):
        y_vars [i] = 0
    constructVars() 
    contructConstrs()
    constructObj() 
    model.write('test.lp')
    model.optimize()
    model.printAttr('X')
    #model.printAttr('Xn')
    model.printAttr('ObjVal')
    model.printAttr('runtime')
    model.printAttr('NodeCount')
    print(f'Time taken to run: {time() - start} seconds')

def constructVars(): 
    global n , m , model, x_vars ,y_vars
    
    for i in range( m ): 
        for j in range( n ): 
            var = model.addVar( vtype = GRB.BINARY , name = "x_" + str( i ) + "_" + str( j ) ) 
            x_vars[ i , j ] = var 
    model.update()
    
    for j in range( n ):
        var_2 = model.addVar( vtype = GRB.BINARY , name = "y_" + str( j )) 
        y_vars[ j ] = var_2
    model.update()

def contructConstrs(): 
    global n , m , model , x_vars , y_vars , P , R
    
    for i in range( m ): 
        constExpr_0 = gp.LinExpr() 
        for j in range( n ): 
            constExpr_0 += 1.0 * x_vars[ i,j ] 
        model.addLConstr( lhs = constExpr_0 , sense = GRB.EQUAL , rhs = '1' ) 
    model.update()
    
    for i in range( m ):
        constExpr_1_1 = gp.LinExpr()
        constExpr_1_2 = gp.LinExpr()
        for j in range( n ):
            constExpr_1_1 = 1.0 * x_vars[i,j]
            constExpr_1_2 = 1.0 * y_vars[j]
            model.addLConstr(lhs= constExpr_1_1, sense = GRB.LESS_EQUAL , rhs = constExpr_1_2)
    model.update()
    
    for j in range( n ):
        constExpr_2 = gp.LinExpr()
        rhs_exp = gp.LinExpr()
        for i in range( m ): 
            constExpr_2 += p[j] * x_vars[ i,j ]
            rhs_exp = P * y_vars[j] 
        model.addLConstr( lhs = constExpr_2 , sense = GRB.LESS_EQUAL , rhs = rhs_exp  ) 
    model.update()
    
    for j in range( n ):
        constExpr_3 = gp.LinExpr()
        rhs_exp = gp.LinExpr()
        for i in range( m ): 
            constExpr_3 += r[j] * x_vars[ i,j ]
            rhs_exp_3 = R * y_vars[j] 
        model.addLConstr( lhs = constExpr_3 , sense = GRB.LESS_EQUAL , rhs = rhs_exp_3  ) 
    model.update()

def constructObj(): 
    global m , n , model , x_vars , y_vars
    objExpr = gp.LinExpr()
    for j in range( n ):
        objExpr += y_vars[j]
    model.setObjective( objExpr , GRB.MINIMIZE ) 
    model.update()

def processing_result(): #source: Gurobi website (https://www.gurobi.com/documentation/9.5/examples/mip2_py.html)
    if len(sys.argv) < 2:
        print('Usage: mip2.py filename')
        sys.exit(0)

    # Read and solve model

    if model.IsMIP == 0:
        print('Model is not a MIP')
        sys.exit(0)

    model.optimize()

    if model.Status == GRB.OPTIMAL:
        print('Optimal objective: %g' % model.ObjVal)
    elif model.Status == GRB.INF_OR_UNBD:
        print('Model is infeasible or unbounded')
        sys.exit(0)
    elif model.Status == GRB.INFEASIBLE:
        print('Model is infeasible')
        sys.exit(0)
    elif model.Status == GRB.UNBOUNDED:
        print('Model is unbounded')
        sys.exit(0)
    else:
        print('Optimization ended with status %d' % model.Status)
        sys.exit(0)

    # Iterate over the solutions and compute the objectives
    model.Params.OutputFlag = 0
    print('')
    for k in range(model.SolCount):
        model.Params.SolutionNumber = k
        print('Solution %d has objective %g' % (k, model.PoolObjVal))
    print('')
    model.Params.OutputFlag = 1

    fixed = model.fixed()
    fixed.Params.Presolve = 0
    fixed.optimize()

    if fixed.Status != GRB.OPTIMAL:
        print("Error: fixed model isn't optimal")
        sys.exit(1)
        
    diff = model.ObjVal - fixed.ObjVal

    if abs(diff) > 1e-6 * (1.0 + abs(model.ObjVal)):
        print('Error: objective values are different')
        sys.exit(1)

#def result(): 
    # Function for printing values of nonzero variables
    #model.optimize()
    #fixed = model.fixed()
    #fixed.Params.Presolve = 0
    #fixed.optimize()
    #for v in model.getVars():
        #if v.X != 0:
            #i = 0
            #answer = ''
            #if v.VarName[0] != 'y':
                #if len(v.VarName) == 7:
                    #if v.Varname[3] != '_':
                        #i += 1
                        #print('job',v.VarName[2:4],'is assigned to machine', v.VarName[5:len(v.VarName)])
                    #else:
                        #i += 1
                        #print('job',v.VarName[2],'is assigned to machine', v.VarName[5:len(v.VarName)])
            
            
                #if len(v.VarName) == 6:
                    #if v.VarName[4] != '_':
                        #i += 1
                        #print('job',v.VarName[2],'is assigned to machine', v.VarName[4:len(v.VarName)])
                    #else:
                        #i += 1
                        #print('job',v.VarName[2:4],'is assigned to machine', v.VarName[len(v.VarName)-1])
            
            
                #if len(v.VarName) == 5:
                    #i += 1
                    #print('job',v.VarName[2],'is assigned to machine',v.VarName[len(v.VarName)-1])

#result() # result of the optimization

runtime = time()
for i in range(10):
    number_of_instance = i
    m , p , r = instance_data( number_of_instance )
    P,R,n = read_data()
    print('\n instnce '+str(number_of_instance)+'\n--------------------')
    Model ( number_of_instance , m , p , r , P , R , n)
    print('\n----------------------------------------------------------------\n\n')
    model.printAttr('NumConstrs')
print(f'Total time taken to run all instances: {time() - runtime} seconds')


 instnce 0
--------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-19
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2650 rows, 2550 columns and 12600 nonzeros
Model fingerprint: 0x33dd0064
Variable types: 0 continuous, 2550 integer (2550 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 38.0000000
Presolve removed 1150 rows and 0 columns
Presolve time: 0.02s
Presolved: 1500 rows, 2550 columns, 7850 nonzeros
Variable types: 0 continuous, 2550 integer (2550 binary)

Root relaxation: objective 1.850000e+01, 1678 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe

## Relaxation:

In [2]:
def contructConstrs(): 
    global n , m , model_2 , x_vars , y_vars , P , R
    
    for i in range( m ): 
        constExpr_0 = gp.LinExpr() 
        for j in range( n ): 
            constExpr_0 += 1.0 * x_vars[ i,j ] 
        model_2.addLConstr( lhs = constExpr_0 , sense = GRB.EQUAL , rhs = '1' ) 
    model_2.update()
    
    for i in range( m ):
        constExpr_1_1 = gp.LinExpr()
        constExpr_1_2 = gp.LinExpr()
        for j in range( n ):
            constExpr_1_1 = 1.0 * x_vars[i,j]
            constExpr_1_2 = 1.0 * y_vars[j]
            model_2.addLConstr(lhs= constExpr_1_1, sense = GRB.LESS_EQUAL , rhs = constExpr_1_2)
    model_2.update()
    
    for j in range( n ):
        constExpr_2 = gp.LinExpr()
        rhs_exp = gp.LinExpr()
        for i in range( m ): 
            constExpr_2 += p[j] * x_vars[ i,j ]
            rhs_exp = P * y_vars[j] 
        model_2.addLConstr( lhs = constExpr_2 , sense = GRB.LESS_EQUAL , rhs = rhs_exp  ) 
    model_2.update()
    
    for j in range( n ):
        constExpr_3 = gp.LinExpr()
        rhs_exp = gp.LinExpr()
        for i in range( m ): 
            constExpr_3 += r[j] * x_vars[ i,j ]
            rhs_exp_3 = R * y_vars[j] 
        model_2.addLConstr( lhs = constExpr_3 , sense = GRB.LESS_EQUAL , rhs = rhs_exp_3  ) 
    model_2.update()
    
    
def constructVars_relax(): 
    global n , m , model_2, x_vars ,y_vars
    
    for i in range( m ): 
        for j in range( n ): 
            var = model_2.addVar( vtype = GRB.CONTINUOUS , name = "x_" + str( i ) + "_" + str( j ) ) 
            x_vars[ i , j ] = var 
    model_2.update()
    
    for j in range( n ):
        var_2 = model_2.addVar( vtype = GRB.CONTINUOUS , name = "y_" + str( j )) 
        y_vars[ j ] = var_2
    model_2.update()

def constructObj(): 
    global m , n , model_2 , x_vars , y_vars
    objExpr = gp.LinExpr()
    for j in range( n ):
        objExpr += y_vars[j]
    model_2.setObjective( objExpr , GRB.MINIMIZE ) 
    model_2.update()
    
def Model( number_of_instance , m , p , r , P , R , n ):
    global model_2, x_vars , y_vars , run_time
    model_2 = gp.Model("q1_instance_"+str(number_of_instance))
    x_vars = {}
    y_vars = {}
    start = time()
    for i in range(n):
        for j in range(m):
            x_vars[i,j] = 0 
    for i in range(m):
        y_vars [i] = 0
    constructVars_relax() 
    contructConstrs()
    constructObj() 
    model_2.write('test.lp')
    model_2.optimize()
    model_2.printAttr('X')
    #model.printAttr('Xn')
    model_2.printAttr('ObjVal')
    model_2.printAttr('runtime')
    model_2.printAttr('NodeCount')
    print(f'Time taken to run: {time() - start} seconds')

runtime = time()
for i in range(10):
    number_of_instance = i
    m , p , r = instance_data( number_of_instance )
    P,R,n = read_data()
    print('\n instnce '+str(number_of_instance)+'\n--------------------')
    Model ( number_of_instance , m , p , r , P , R , n)
    print('\n----------------------------------------------------------------\n\n')

print(f'Total time taken to run all instances: {time() - runtime} seconds')


 instnce 0
--------------------
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2650 rows, 2550 columns and 12600 nonzeros
Model fingerprint: 0xb998cd5f
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 2649 rows, 2550 columns, 12549 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.709375e+01   0.000000e+00      0s
      52    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

      50 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase complete: Pinf 0.0000000e+0